# 05. Filtración de los datos

---

Michael Heredia Pérez - mherediap@unal.edu.co - Universidad Nacional de Colombia Sede Manizales

---

En este cuaderno se presentan algunos comandos básicos para el uso de filtros desde ```pvpython```.



In [ ]:
from paraview.simple import *

Para crear un filtro se crea el objeto usando su nombre como una función ```Filtro()``` y este se aplicará sobre la fuente activa. En el argumento de la función se puede especificar la fuente sobre la cual se quiere aplicar.

In [ ]:
filtro = Shrink() # Filtro aplicadosobre la fuente activa atuomáticamente.
filtro = Shrink(Input = reader) # Filtro aplicado sobre la fuente "reader".

### Multiples conexiones

Recordemos que la fuente de un filtro puede venir de múltiples conexiones, para lograr estas conexiones múltiples se deben pasar las fuentes como una lista en el argumento del filtro ```AppendDatasets()```

In [ ]:
# Las fuentes que se emplean
fuente_1 = Sphere()
fuente_2 = Cone()
fuente_3 = Cylinder()

conexion_mult = AppendDatasets(Input = [fuente_1, fuente_2, fuente_3])

### Multiples fuentes de entrada

Se componen las fuentes de entrada mediante la función ```ResampleWithDataSet()```. **Hay que tener cuidado con esta función** ya que su uso no está actualizado en el libro guia ParaViewGuide-5.8.1.

Si creamos una fuente nueva


In [ ]:
fuente_4 = Wavelet()

El manual diría que los argumentos de la función para componer la ```fuente_4``` con la ```fuente_3```, por ejemplo, sería de la siguiente forma:

In [ ]:
remuestreo_con_datos = ResampeWithDataset(Input = fuente_3, Source = fuente_4)

Sin embargo, [desde la versión 5.7](https://blog.kitware.com/paraview-5-7-0-release-notes/) de ParaView estos argumentos fueron cambiados por los siguientes:

In [ ]:
remuestreo_con_datos = ResampeWithDataset(SourceDataArrays = fuente_3, 
                                          DestinationMesh = fuente_4)

El cambio es:

| Antes de la versión 5.7 | Después de la versión 5.7 |
| ---| --- |
| ```Input = ``` | ```SourceDataArrays = ``` |
| ```Source = ``` | ```DestinationMesh = ``` | 

### Modificar las conexiones de entrada

Es como si se cambiara una propiedad en el filtro. 

Si la fuente ```fuente``` es a la cual queremos cambiar entonces en un filtro con una única entrada bastará con cambiar la propiedad ```Input```:

In [ ]:
filtro = Shrink()       # Aplicado sobre la fuente activa
filtro.Input = fuente 

Para filtros con entradas de conexiones multiples

In [ ]:
conexion_mult.Input = [fuente_3, fuente_4]

Para filtros con múltiples fuentes de entradas

In [ ]:
remuestro_con_datos.SourceDataArrays = fuente_1     # Input
remuestreo_con_datos.DestinationMesh = fuente_2     # Source

## Cambiar propiedades de los filtros

La modificaciones de las propiedades es accesible desde ```pvpython``` de igual forma a como se ha visto en otras secciones y con un procedimiento similar al de cambiar la fuente de entrada.

Para conocer las propiedades disponibles para ser modificadas de un filtro, se puede emplear el comando ```help()```.

## Filtros para subgrupos de datos.

### ```Clip```

Algunos comandos sobre el filtro ```Clip```.

In [ ]:
# Para crear el filtro.
filtro_clip = Clip(Input = ...)

# Para especificar el tipo de Clip o "ClipType"
filtro_clip.ClipType('Plane')

# o mediante la API de propiedades
SetProperties(filtro_clip, ClipType = 'Plane')

# Para conocer los tipos de clip disponibles
filtro_clip.GetProperty('ClipType').GetAvailable()

# Para establecer el origen y vector normal del plano
filtro_clip.ClipType.Origin = [0, 0, 0]
filtro_clip.ClipType.Normal = [1, 0, 0]

# Si se quisiera cambiar a una esfera como tipo de clip y establecer sus 
# propiedades.
filtro_clip.ClipType = 'Sphere'
filtro_clip.ClipType.Center = [0, 0, 0]
filtro_clip.ClipType.Radius = 12
# De igual forma se podría a través de la API de propiedades.

# Para establecer el valor el Crinkle clipping
filtro_clip.Crinkleclip = 1

# Para hacer clip con un escalar
filtro_clip.ClipType = 'Scalar'
filtro_clip.Scalar = ('POINTS', 'Temp')     # El tipo y el arreglo.
filtro_clip = 100

## Filtros para modificación geométrica (*geometric manipulation*)

### ```Transform```

Con este filtro se hace un traslado, rotación o escalamiento de la geometría. Para crearlo desde ```pvpython``` se especifica la entrad, o de no hacerlo se tomará la fuente activa como entrada; los comandos son los siguientes:

In [ ]:
# Se aplica el filtro.
transformando = Transform(Input = ...)

# Se modifican las propiedades del filtro.
transformando.Transforme.Scale = [1, 2, 1]
transformando.Transforme.Translate = [100, 0, 0]
transformando.Transforme.Rotate = [0, 0, 0]        # D igual si s epusiera o no.

## Filtros para muestreo (*sampling*)

### ```Stream Tracer```

Este filtro produce líneas de flujo en un cmpo vectorial, y e eejmplo perfecto desde los archivos ejemplo de paraview son es el modelado ```disk_out_refex2.ex2```.

La sintaxis para crear el filtro con este ejemplo es la siguiente:



In [ ]:
# Se crea la fuente.
disk_out_refex2 = FindSource('disk_out_ref.ex2')

# Se crea el filtro Stream Tracer.
filtro_stream_tracer = SteamTracer(Input = disk_out_refex2,
                                   SeedType = 'Point Source')
filtro_stream_tracer.Vectors = ['Points', 'V']

# Se inicia la semilla.
filtro_stream_tracer.SeedType.Center = [0, 0, 0.079]  # Random.
filtro_stream_tracer.SeedType.Radius = 2.015

# Se muestran los datos en la vista.
Show()

# Se aplica Tube Filter.
filtro_tube = Tube(Input = filtro_stream_tracer)    # El input del filtro Tube 
                                                    # será la salida del filtro 
                                                    # Stream Tracer.

# Se modifican algunas propiedades del filtro y se muestra.
filtro_tube.Radius = 0.16       # Se podría dar en función de otro valor.
Show()